In [2]:
import pandas as pd

In [42]:
df = pd.read_excel('chunk.xlsx')

In [43]:
df.head()

,Titulo,Contenido,Autor,Año,Ciudad,Clasificación
0,COP 16 Cali,No paraba de mover su mano derecha con un lápi...,El Colombiano,2023,Cali,Medio ambiente
1,Muerte de Ze Carlos,"El exlateral Zé Carlos, subcampeón con Brasil ...",El Espectador,2024,San Pablo,Deporte
2,Destino gastronómico mundial,Me encanta la frase tan paisa y pintoresca que...,El Tiempo,2024,Medellín,Economía
3,La caída del GEA,"Dos compañías independientes que, en la medida...",Semana,2023,Medellín,Economía
4,Polémica en el RTVC,"Una vez más, la información que divulga el Sis...",El Colombiano,2023,Bogotá,Política


In [44]:
def clean_text(df):
    df = df.replace('á','a', regex=True)
    df = df.replace('Á','A', regex=True)
    df = df.replace('é','e', regex=True)
    df = df.replace('É','E', regex=True)
    df = df.replace('Í','I', regex=True)
    df = df.replace('í','i', regex=True)
    df = df.replace('ó','o', regex=True)
    df = df.replace('Ó','O', regex=True)
    df = df.replace('ú','u', regex=True)
    df = df.replace('Ú','U', regex=True)
    return df

In [45]:
df = clean_text(df)

In [27]:
def merge_columns_with_names(df: pd.DataFrame) -> pd.Series:
    
    def combine_row(row):
        filtered_items = {k: v for k, v in row.items()}
        
        combined = ", ".join([f"{col}: {str(val).strip()}" for col, val in filtered_items.items()])
        return combined
    
    return df.apply(combine_row, axis=1)

In [28]:
df['Combinado'] = merge_columns_with_names(df)

In [29]:
df.head()

,Titulo,Contenido,Autor,Año,Ciudad,Clasificación,Combinado
0,COP 16 Cali,No paraba de mover su mano derecha con un lápi...,El Colombiano,2023,Cali,Medio ambiente,"Titulo: COP 16 Cali, Contenido: No paraba de m..."
1,Muerte de Ze Carlos,El exlateral Zé Carlos subcampeón con Brasil e...,El Espectador,2024,San Pablo,Deporte,"Titulo: Muerte de Ze Carlos, Contenido: El exl..."
2,Destino gastronómico mundial,Me encanta la frase tan paisa y pintoresca que...,El Tiempo,2024,Medellín,Economía,"Titulo: Destino gastronómico mundial, Contenid..."
3,La caída del GEA,Dos compañías independientes que en la medida ...,Semana,2023,Medellín,Economía,"Titulo: La caída del GEA, Contenido: Dos compa..."
4,Polémica en el RTVC,Una vez más la información que divulga el Sist...,El Colombiano,2023,Bogotá,Política,"Titulo: Polémica en el RTVC, Contenido: Una ve..."


In [36]:
len(df.iloc[2,2])

9

In [40]:
def contar_palabras(texto):
    return len(texto.split())

texto_ejemplo = "Este es un   texto de\nejemplo\tcon espacios"
print(contar_palabras(texto_ejemplo))

8


In [41]:
import pandas as pd
import math

def dividir_contenido_largo(df: pd.DataFrame, max_chars: int = 100) -> pd.DataFrame:
    nuevas_filas = []
    
    for _, fila in df.iterrows():
        contenido = str(fila['Contenido'])
        
        if len(contenido) <= max_chars:
            nuevas_filas.append(fila.to_dict())
            continue
            
        palabras = contenido.split()
        parte_actual = []
        longitud_actual = 0
        
        for palabra in palabras:
            longitud_palabra = len(palabra) + 1
            
            if longitud_actual + longitud_palabra > max_chars and parte_actual:
                nueva_fila = fila.to_dict()
                nueva_fila['Contenido'] = ' '.join(parte_actual).strip()
                nuevas_filas.append(nueva_fila)
                
                parte_actual = [palabra]
                longitud_actual = longitud_palabra
            else:
                parte_actual.append(palabra)
                longitud_actual += longitud_palabra
        
        if parte_actual:
            nueva_fila = fila.to_dict()
            nueva_fila['Contenido'] = ' '.join(parte_actual).strip()
            nuevas_filas.append(nueva_fila)
    
    return pd.DataFrame(nuevas_filas)

In [56]:
df_resultado = dividir_contenido_largo(df, 800)

In [57]:
df_resultado

,Titulo,Contenido,Autor,Año,Ciudad,Clasificación
0,COP 16 Cali,No paraba de mover su mano derecha con un lápi...,El Colombiano,2023,Cali,Medio ambiente
1,Muerte de Ze Carlos,El exlateral Zé Carlos subcampeón con Brasil e...,El Espectador,2024,San Pablo,Deporte
2,Destino gastronómico mundial,Me encanta la frase tan paisa y pintoresca que...,El Tiempo,2024,Medellín,Economía
3,Destino gastronómico mundial,restauración es un campo que exige conocimient...,El Tiempo,2024,Medellín,Economía
4,La caída del GEA,Dos compañías independientes que en la medida ...,Semana,2023,Medellín,Economía
5,Polémica en el RTVC,Una vez más la información que divulga el Sist...,El Colombiano,2023,Bogotá,Política
6,J Balvin en Manrique,J Balvin ha tenido una semana bastante movida ...,Semana,2022,Medellín,Entretenimiento


In [93]:
import pandas as pd

def chunk_dataframe_text(df, max_chunks, overlap):
    result_df = df.copy()
    
    def combine_row(row):
        return ', '.join([f"{col}: {str(val)}" for col, val in row.items() if pd.notna(val)])
    
    result_df['combinado'] = result_df.apply(combine_row, axis=1)
    
    def split_text_into_chunks(text, max_chunks, overlap):
        words = text.split()
        if len(words) <= max_chunks:
            return [text]
        
        chunks = []
        current_position = 0
        
        while current_position < len(words):
            end_position = min(current_position + max_chunks, len(words))
            
            current_chunk = ' '.join(words[current_position:end_position])
            chunks.append(current_chunk)
            
            if end_position < len(words):
                current_position = end_position - overlap
                
                remaining_words = len(words) - current_position
                if remaining_words < max_chunks // 2:
                    chunks[-1] = ' '.join(words[current_position - overlap:])
                    break
            else:
                break
        
        return chunks

    new_rows = []
    next_index = 0 
    
    for _, row in result_df.iterrows():
        text = row['combinado']
        chunks = split_text_into_chunks(text, max_chunks, overlap)

        if len(chunks) == 1:
            row_dict = row.to_dict()
            row_dict['original_index'] = next_index
            new_rows.append(row_dict)
            next_index += 1
        else:
            for chunk in chunks:
                row_dict = row.to_dict()
                row_dict['combinado'] = chunk
                row_dict['original_index'] = next_index
                new_rows.append(row_dict)
                next_index += 1
    
    final_df = pd.DataFrame(new_rows)
    final_df = final_df.reset_index(drop=True)
    
    if 'original_index' in final_df.columns:
        final_df = final_df.drop('original_index', axis=1)
    
    return final_df

In [94]:
df_resultado = chunk_dataframe_text(df, max_chunks=100, overlap=10)

In [95]:
df_resultado

,Titulo,Contenido,Autor,Año,Ciudad,Clasificación,combinado
0,COP 16 Cali,No paraba de mover su mano derecha con un lápi...,El Colombiano,2023,Cali,Medio ambiente,"Titulo: COP 16 Cali, Contenido: No paraba de m..."
1,Muerte de Ze Carlos,El exlateral Zé Carlos subcampeón con Brasil e...,El Espectador,2024,San Pablo,Deporte,"Titulo: Muerte de Ze Carlos, Contenido: El exl..."
2,Destino gastronómico mundial,Me encanta la frase tan paisa y pintoresca que...,El Tiempo,2024,Medellín,Economía,"Titulo: Destino gastronómico mundial, Contenid..."
3,Destino gastronómico mundial,Me encanta la frase tan paisa y pintoresca que...,El Tiempo,2024,Medellín,Economía,terrible costumbre de pensar que sabemos de to...
4,La caída del GEA,Dos compañías independientes que en la medida ...,Semana,2023,Medellín,Economía,"Titulo: La caída del GEA, Contenido: Dos compa..."
5,Polémica en el RTVC,Una vez más la información que divulga el Sist...,El Colombiano,2023,Bogotá,Política,"Titulo: Polémica en el RTVC, Contenido: Una ve..."
6,J Balvin en Manrique,J Balvin ha tenido una semana bastante movida ...,Semana,2022,Medellín,Entretenimiento,"Titulo: J Balvin en Manrique, Contenido: J Bal..."


In [100]:
def df_to_row_json(df):
    json_data = {}
    
    for i, row in df.iterrows():
        chunk_key = f'chunk{i+1}'
        

        row_dict = {}
        for column, value in row.items():
            if pd.isna(value):
                value = None
            elif isinstance(value, (pd.Timestamp, pd.DatetimeIndex)):
                value = value.strftime('%Y-%m-%d')
            elif hasattr(value, 'item'):
                value = value.item()
                
            row_dict[column] = value
        
        json_data[chunk_key] = row_dict
    
    return json_data

In [102]:
json_r = df_to_row_json(df_resultado)

In [103]:
json_r

{'chunk1': {'Titulo': 'COP 16 Cali',
  'Contenido': 'No paraba de mover su mano derecha con un lápiz golpeando la mesa Con tono de resignación el presidente Gustavo Petro soltó una frase que dejó en silencio al auditorio ocupado por representantes de varios países que asistieron a la Conferencia Interamericana de Ministras y Ministros de Trabajo de la OEA En este momento se está realizando la COP16 en Cali Dice el New York Times que es el más importante de la historia en biodiversidad se han reunido los pueblos los gobernantes no cierto Ni siquiera Lula me acompañó pero bueno',
  'Autor': 'El Colombiano',
  'Año': 2023,
  'Ciudad': 'Cali',
  'Clasificación': 'Medio ambiente',
  'combinado': 'Titulo: COP 16 Cali, Contenido: No paraba de mover su mano derecha con un lápiz golpeando la mesa Con tono de resignación el presidente Gustavo Petro soltó una frase que dejó en silencio al auditorio ocupado por representantes de varios países que asistieron a la Conferencia Interamericana de Minis

In [5]:
from xlsxchunk import chunkgenerator

json_r = chunkgenerator('chunk.xlsx',None,None)
json_r

{'chunk1': {'Titulo': 'COP 16 Cali',
  'Contenido': 'No paraba de mover su mano derecha con un lapiz golpeando la mesa. Con tono de resignacion, el presidente Gustavo Petro solto una frase que dejo en silencio al auditorio ocupado por representantes de varios paises que asistieron a la Conferencia Interamericana de Ministras y Ministros de Trabajo de la OEA: “En este momento se esta realizando la COP16 en Cali. Dice el New York Times que es el mas importante de la historia en biodiversidad, se han reunido los pueblos, los gobernantes no, ¿cierto? Ni siquiera Lula me acompaño, pero bueno”.',
  'Autor': 'El Colombiano',
  'Año': 2023,
  'Ciudad': 'Cali',
  'Clasificación': 'Medio ambiente',
  'Combinado': 'Titulo: COP 16 Cali, Contenido: No paraba de mover su mano derecha con un lapiz golpeando la mesa. Con tono de resignacion, el presidente Gustavo Petro solto una frase que dejo en silencio al auditorio ocupado por representantes de varios paises que asistieron a la Conferencia Interame